## Swarm Simulation 

This is for testing propouses

In [1]:
import os
import subprocess
from pymavlink import mavutil

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns
from helpers.gazebo_world import update_world

## Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

## Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/attack.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

# Choose Simulator

In [4]:
simulator ='gazebo' #'QGroundControl' # 

# Choose Initial Position

In [5]:
offsets = [ #east, north, up, heading
    (20, -10, 0, 0)]
n_uavs = len(offsets)

                # lat,long,alt,heading
if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,90)
    spawns=find_spawns(home_position, offsets)

## Launch Simulator

In [6]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(f"drone{i+1}", east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

## Waypoint seletion algorithm

In [7]:
wps_2d=[(0, 0), #takeoff point
        (0, -20),
        (-10,0),
        (-10, -20),
        (-20, 0),
        (-20, -20),
        (-20, -10)]


def find_best_move(current_position, target_position, possible_points):
    x_cur, y_cur = current_position
    x_target, y_target = target_position

    # Manhattan distance function
    def manhattan_distance(p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

    # Function to check if a waypoint is in the correct quadrant
    def in_same_quadrant(waypoint):
        x_wp, y_wp = waypoint
        return (
            (x_target - x_cur) * (x_target - x_wp) >= 0 and  # X direction check
            (y_target - y_cur) * (y_target - y_wp) >= 0      # Y direction check
        )

    # Filter points that share x or y with the target AND are in the correct quadrant
    valid_moves = [
        p for p in possible_points
        if (abs(p[0] -x_cur)<2) or (abs(p[1]<y_cur)<2) and in_same_quadrant(p)
    ]

    if not valid_moves:
        return None  # No valid moves available

    # Find the move that gets closest to the target
    best_move = min(valid_moves, key=lambda p: manhattan_distance(p, current_position))

    return best_move

def find_path(start, target, waypoints):
    path = [start]
    current_position = start

    while current_position != target:
        next_move = find_best_move(current_position, target, waypoints)
        if next_move is None:
            break  # No valid path found
        path.append(next_move)
        waypoints.remove(next_move)  # Remove the used waypoint
        current_position = next_move

    return path

# Define start, target, and possible waypoints
start_pos = wps_2d[0]
target_pos = wps_2d[-1]
possible_waypoints = wps_2d[1:]

# Compute the path


#path_2d = find_path(start_pos, target_pos, possible_waypoints)
#path_2d


## Create Plan

In [8]:
plan=['check_prearm','check_pos_est','mode_stabilize','mode_guided','arm','takeoff','fly','land']
uavs_arg = []
altitude = 5
wps1=[(0, 0,-altitude), #takeoff point
    (-10,0, -altitude),
    (-20, 0, -altitude),
    (-20, -10, -altitude)]

wps2=[(0, 0,-altitude), #takeoff point
    (-10,0, -altitude),
    (-10, -20, -altitude),
    (-20, -20, -altitude),
    (-20, -10, -altitude)]
path_2d=find_path(start_pos, target_pos, possible_waypoints.copy())
path = [(x, y, -altitude) for x, y in path_2d]
for i in range(n_uavs):

    uav_arg={'waypoints':path,
             'plan': plan,
             'wp_margin': 0.5}
    uavs_arg.append(uav_arg)

# Launch Vehicle

In [9]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)

In [10]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

## Create UavLogic

In [11]:
uavs=[]
for i,uav_arg in enumerate(uavs_arg):
    connection = mavutil.mavlink_connection(f'udp:127.0.0.1:{14551+10*i}')
    connection.wait_heartbeat()
    uav=VehicleLogic(connection,**uav_arg)
    uavs.append(uav)

vehicle 1 created


## Execute Plan

In [12]:
target_pos

(-20, -10)

In [13]:
possible_waypoints

[(0, -20), (-10, 0), (-10, -20), (-20, 0), (-20, -20), (-20, -10)]

In [14]:
possible_waypoints

[(0, -20), (-10, 0), (-10, -20), (-20, 0), (-20, -20), (-20, -10)]

In [15]:
uav.wp_i

0

In [16]:
path

[(0, 0, -5), (-10, 0, -5), (-20, 0, -5), (-20, -10, -5)]

In [17]:
mission_incomplete = n_uavs*[True]
while any(mission_incomplete):
    for i,uav in enumerate(uavs):
        mission_incomplete[i] = uav.act_plan()
        if uav.action_i<uav.n_actions and uav.plan[uav.action_i]=='fly' and uav.is_reached:
            current_position= uav.get_position()[:2]
            path_2d = find_path(current_position, target_pos, possible_waypoints)
            path = [(x, y, -altitude) for x, y in path_2d]
            uav.update_waypoints(path)
        

0
land
vehicle 1: action land is done
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0
check_prearm
0


## Kill all related process

In [18]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")